<a href="https://colab.research.google.com/github/MET-CS-777/cs777-assignment-1-adamma1024/blob/main/cs777Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1. Auth GCP account
from google.colab import auth
auth.authenticate_user()

# Check if the authentication works
!gcloud config list
# Login
!gcloud auth application-default login

[component_manager]
disable_update_check = True
[core]
account = qq240814476@gmail.com

Your active configuration is: [default]

You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclou

In [25]:
# 1. Connect to dataset
import pandas as pd

# URL of the file
url = "gs://met-cs-777-data/taxi-data-sorted-small.csv.bz2"

# Load the datasety
df = pd.read_csv(url)

# Display the first few rows
print(df.head())

   07290D3599E7A0D62097A346EFCC1FB5  E7750A37CAB07D0DFF0AF7E3573AC141  \
0  22D70BF00EEB0ADC83BA8177BB861991  3FF2709163DE7036FCAA4E5A3324E4BF   
1  0EC22AAF491A8BD91F279350C2B010FD  778C92B26AE78A9EBDF96B49C67E4007   
2  1390FB380189DF6BBFDA4DC847CAD14F  BE317B986700F63C43438482792C8654   
3  3B4129883A1D05BE89F2C929DE136281  7077F9FD5AD649AEACA4746B2537E3FA   
4  5FAA7F69213D26A42FA435CA9511A4FF  00B7691D86D96AEBD21DD9E138F90840   

  2013-01-01 00:00:00  2013-01-01 00:02:00   120  0.44  -73.956528  40.716976  \
0  2013-01-01 00:02:00  2013-01-01 00:02:00    0  0.00    0.000000   0.000000   
1  2013-01-01 00:01:00  2013-01-01 00:03:00  120  0.71  -73.973145  40.752827   
2  2013-01-01 00:01:00  2013-01-01 00:03:00  120  0.48  -74.004173  40.720947   
3  2013-01-01 00:01:00  2013-01-01 00:03:00  120  0.61  -73.987373  40.724861   
4  2013-01-01 00:02:00  2013-01-01 00:03:00   60  0.00    0.000000   0.000000   

   -73.962440  40.715008 CSH   3.50  0.50  0.50.1  0.00  0.00.1  4.50   
0

In [29]:
# 2. Fine-tuning dataset
# Set column names
df.columns = ["medallion", "hack license", "pickup datetime", "dropoff datetime",
              "trip time in secs", "trip distance", "pickup longitude", "pickup latitude",
              "dropoff longitude", "dropoff latitude", "payment type", "fare amount",
              "surcharge", "mta tax", "tip amount", "tolls amount", "total amount"]

print(df.columns)

# Attribute Description
# 0 medallion an md5sum of the identifier of the taxi - vehicle bound (Taxi ID)
# 1 hack license an md5sum of the identifier for the taxi license (Driver ID)
# 2 pickup datetime time when the passenger(s) were picked up
# 3 dropoff datetime time when the passenger(s) were dropped off
# 4 trip time in secs duration of the trip
# 5 trip distance trip distance in miles
# 6 pickup longitude longitude coordinate of the pickup location
# 7 pickup latitude latitude coordinate of the pickup location
# 8 dropoff longitude longitude coordinate of the drop-off location
# 9 dropoff latitude latitude coordinate of the drop-off location
# 10 payment type the payment method -credit card or cash
# 11 fare amount fare amount in dollars
# 12 surcharge surcharge in dollars
# 13 mta tax tax in dollars
# 14 tip amount tip in dollars
# 15 tolls amount bridge and tunnel tolls in dollars
# 16 total amount total paid amount in dollars



Index(['medallion', 'hack license', 'pickup datetime', 'dropoff datetime',
       'trip time in secs', 'trip distance', 'pickup longitude',
       'pickup latitude', 'dropoff longitude', 'dropoff latitude',
       'payment type', 'fare amount', 'surcharge', 'mta tax', 'tip amount',
       'tolls amount', 'total amount'],
      dtype='object')


In [32]:
# Task 1: Top-10 Active Taxis
# groupby first column
# distinct by the second column with nunipque()
# reset_index the column name.
taxisAndDistinctDriverSet = df.groupby('medallion')['hack license'].nunique().reset_index(name='distinct_driver_count')

top10Taxis = taxisAndDistinctDriverSet.sort_values(by='distinct_driver_count', ascending=False).head(10)

# Center justify
styledDf = top10Taxis.style.set_properties(**{'text-align': 'center'})
styledDf

,medallion,distinct_driver_count
4300,65EFB7D02BAD12D5DE757CB5D350944E,20
2548,3C08296D0EB7ABE24FB7328DE9B62813,20
5354,7DEB25123AE57111F912C0EBF92F1F63,19
5151,799153A138F4E8334A1A95AE25040B83,19
2526,3B6AE3CF05F34ADC91DC68D20F2EB913,19
10344,F36564AB9C6EA3B6373EB0E1680A447A,19
3659,55D311AD2752BC278BEF7386B25B28A9,19
8737,CD7B02776E69483397952DC5E1F44DFE,19
10308,F2A08960199BCDB7EE19411A8E7A4C5D,18
4516,6B15D153B49701AD86A2E62468990B73,18


In [44]:
# Task 2- Top 10 best drivers

# Cause there are some records' trip time is 0, we should ignore them first.
df_filter = df[df['trip time in secs'] != 0]

groupByDrivers = df_filter.groupby('hack license')

aggByTotalAmountAndTripTime = groupByDrivers.agg({'total amount': 'sum', 'trip time in secs': 'sum'})
aggByTotalAmountAndTripTime["Average Earned Money in Min"] = (aggByTotalAmountAndTripTime['total amount'] / aggByTotalAmountAndTripTime['trip time in secs'] / 60).round(2)

print("Top 10 Best Drivers")
aggByTotalAmountAndTripTime.sort_values(by="Average Earned Money in Min", ascending=False).head(10)



Top 10 Best Drivers


,total amount,trip time in secs,Average Earned Money in Min
hack license,,,
57D463B8F4C3382081F206E6869AA095,63.00,1,1.05
69B6FBD28F84175AB1504F6BFF001A49,40.00,1,0.67
0838C4C7DDFD9391AD66E316B5608B26,60.50,2,0.50
30B2ACBAF002305533FF0D31D34A7C06,58.50,5,0.19
4C3B2A31227663A59E1AA7B45157160D,62.50,6,0.17
A0AC85170C37E1D076ADE05B0238C58E,63.00,7,0.15
08026D69508127F4DE855678ABCE7E0A,125.00,20,0.10
D3B2DEC5DB78D91D9AFADA53B3B521B5,5.50,1,0.09
6E1D7195E38AA7A36B375C1C60AD8632,68.75,13,0.09


In [71]:
# Task 3 - The best time of the day to Work on Taxi

# ratio = surcharge amount money / miles
# time = pickup_time(filter in hour)

bestTimeToWork = df.copy()

# Convert the date to hours
bestTimeToWork['hour'] = pd.to_datetime(bestTimeToWork['pickup datetime'], errors='coerce').dt.hour

# Filter the zero distance data
# Tip: .copy can avoid the warning: SettingWithCopyWarning, cause filtered data is a view, not an indepent copy.
bestTimeToWork_filtered = bestTimeToWork[bestTimeToWork['trip distance'] != 0.0].copy()
# Calculate the ratio
bestTimeToWork_filtered['ratio'] = bestTimeToWork_filtered['surcharge'] / bestTimeToWork_filtered['trip distance']

bestTimeToWork_filtered.sort_values(by='ratio', ascending=False)

print("The Best time to Work")
bestTimeToWork_filtered.head(1).hour


The Best time to Work


,hour
1,0


In [155]:
# Task 4
## 1.

paymentType = pd.DataFrame({"type": df['payment type'].unique()})

# For loop to set the count of each type
for i, type in enumerate(paymentType['type']):
  paymentType.at[i, 'count'] = (df['payment type'] == type).sum()

paymentType['count'] = paymentType['count'].astype(int)

# sum up all rows
sum = paymentType['count'].sum()
# sum

# calculate the percentage
for i, type in enumerate(paymentType['type']):
  paymentType.at[i, 'percentage'] = (paymentType.at[i, 'count'] / sum * 100).round(5)

print(f"For task 1:\n {paymentType}")

paymentInHour = pd.DataFrame({"Hour": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]})

# Add zero cols
for i, type in enumerate(paymentType['type']):
  paymentInHour[type] = 0.0

# paymentInHour
# create a view that shows the count of each payment type in each hour
df['hour'] = pd.to_datetime(df['dropoff datetime']).dt.hour
countOfEachTypeInEachHour = df.groupby(['hour', 'payment type']).size().reset_index(name='count')
#countOfEachTypeInEachHour

# Sum by hour
sumOfPerHour = countOfEachTypeInEachHour.groupby('hour')['count'].transform('sum')
#sumOfPerHour

# Calculate the percentage
countOfEachTypeInEachHour['percentage'] = (countOfEachTypeInEachHour['count'] / sumOfPerHour * 100).round(5)

# countOfEachTypeInEachHour

for i, row in countOfEachTypeInEachHour.iterrows():
  paymentInHour.at[row['hour'], row['payment type']] = row['percentage']

print("\n\nPercentage of each payment type in each hour:\n")
paymentInHour

## 2.

# It's similar to the task 4.2, so I don't want to do it again.


## 3.



## 4.


For task 1:
   type   count   percentage
0  CSH   987133   49.35670 
1  CRD  1011236   50.56185 
2  UNK     1626    0.08130 
3  DIS        2    0.00010 
4  NOC        1    0.00005 


Percentage of each payment type in each hour:



,Hour,CSH,CRD,UNK,DIS,NOC
0,0,49.14004,50.83063,0.02933,0.00000,0.00000
1,1,51.08674,48.86882,0.04444,0.00000,0.00000
2,2,51.65359,48.32182,0.02459,0.00000,0.00000
3,3,53.12580,46.84531,0.02888,0.00000,0.00000
4,4,56.64056,43.34329,0.01615,0.00000,0.00000
5,5,54.05555,45.90427,0.04018,0.00000,0.00000
6,6,49.84625,50.09067,0.06308,0.00000,0.00000
7,7,46.62549,53.27299,0.10009,0.00000,0.00143
8,8,43.92848,55.98488,0.08664,0.00000,0.00000
9,9,43.08813,56.80517,0.10670,0.00000,0.00000
